# Supermarket Sales Analysis
This notebook conducts a comprehensive analysis of supermarket sales data stored in a MySQL database.

In [1]:
import mysql.connector as connector
import pandas as pd
from dotenv import load_dotenv
import os

## Database Connection and Setup
This section connects to the MySQL database, creates the `supermarket_sales` database and the `Sales` table if they do not exist.

In [2]:
# Load environment variables from .env file
load_dotenv()
# Configuration for the database connection
dbconfig = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}

# Connect to the database
connection = connector.connect(**dbconfig)
cursor = connection.cursor()

In [3]:
# Create the supermarket_sales databes if it does not exist
db_create_query = "CREATE DATABASE IF NOT EXISTS supermarket_sales;"
db_use_query = "USE supermarket_sales;"
cursor.execute(db_create_query)
cursor.execute(db_use_query)

# Check if the database is created and in use
check_query = "SELECT DATABASE() AS current_database;"
cursor.execute(check_query)
result = cursor.fetchone()

if result[0] == "supermarket_sales":
    print("Database created and in use: supermarket_sales")
else:
    print("Database not created or not in use")

Database created and in use: supermarket_sales


In [4]:
table_create_query = """
CREATE TABLE IF NOT EXISTS sales (
    invoice_id VARCHAR(20),
    branch CHAR(1),
    city VARCHAR(50),
    customer_type VARCHAR(10),
    gender VARCHAR(10),
    product_line VARCHAR(50),
    unit_price DECIMAL(10, 2),
    quantity INT,
    tax DECIMAL(10, 2),
    total DECIMAL(10, 2),
    date DATE,
    time TIME,
    payment VARCHAR(20),
    cogs DECIMAL(10, 2),
    gross_margin_percentage DECIMAL(5, 2),
    gross_income DECIMAL(10, 2),
    rating DECIMAL(4, 2)
);
"""
cursor.execute(table_create_query)
print("Sales table created or already exists.")


# Check if the table is created
check_query = "SHOW TABLES LIKE 'sales';"
cursor.execute(check_query)
result = cursor.fetchall()
if result:
    print("Sales table exists.")
else:
    print("Sales table does not exist.")


Sales table created or already exists.
Sales table exists.


## Data Insertion
This section reads sales data from a CSV file and inserts it into the `Sales` table in the MySQL database.

In [5]:
# Read the CSV file using Pandas
df = pd.read_csv('../data/supermarket_sales.csv')

# Convert the 'date' column to datetime and format it
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

# Clip the 'rating' values to ensure they are within the 0 to 10.00 range
df['Rating'] = df['Rating'].clip(0, 10.00)

# Insert data into the sales table
insert_query = """
    INSERT INTO sales (invoice_id, branch, city, customer_type, gender, product_line, unit_price, quantity, tax, total, date, time, payment, cogs, gross_margin_percentage, gross_income, rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
data = [tuple(row) for row in df.itertuples(index=False, name=None)]
cursor.executemany(insert_query, data)

# Commit the transaction
connection.commit()

# Check if the data is inserted
check_query = "SELECT * FROM sales LIMIT 10;"
cursor.execute(check_query)
result = cursor.fetchall()
for row in result:
    print(row)

('750-67-8428', 'A', 'Yangon', 'Member', 'Female', 'Health and beauty', Decimal('74.69'), 7, Decimal('26.14'), Decimal('548.97'), datetime.date(2019, 1, 5), datetime.timedelta(seconds=47280), 'Ewallet', Decimal('522.83'), Decimal('4.76'), Decimal('26.14'), Decimal('9.10'))
('226-31-3081', 'C', 'Naypyitaw', 'Normal', 'Female', 'Electronic accessories', Decimal('15.28'), 5, Decimal('3.82'), Decimal('80.22'), datetime.date(2019, 3, 8), datetime.timedelta(seconds=37740), 'Cash', Decimal('76.40'), Decimal('4.76'), Decimal('3.82'), Decimal('9.60'))
('631-41-3108', 'A', 'Yangon', 'Normal', 'Male', 'Home and lifestyle', Decimal('46.33'), 7, Decimal('16.22'), Decimal('340.53'), datetime.date(2019, 3, 3), datetime.timedelta(seconds=48180), 'Credit card', Decimal('324.31'), Decimal('4.76'), Decimal('16.22'), Decimal('7.40'))
('123-19-1176', 'A', 'Yangon', 'Member', 'Male', 'Health and beauty', Decimal('58.22'), 8, Decimal('23.29'), Decimal('489.05'), datetime.date(2019, 1, 27), datetime.timedelta

## Data Analysis
This section contains various SQL queries to analyze the sales data. The results are displayed as Pandas DataFrames for better readability.

In [6]:
def query_to_dataframe(cursor, result):
    """
    Convert SQL query result to a pandas DataFrame and display it.
    
    Parameters:
    cursor: Database cursor with description attribute
    result: Result set from the executed query

    Returns:
    df_result: pandas DataFrame containing the query results
    """
    # Get the column names from the cursor description
    columns = [desc[0] for desc in cursor.description]
    
    # Create a DataFrame from the fetched results
    df_result = pd.DataFrame(result, columns=columns)
    
    # Display the DataFrame
    print(df_result)
    
    return df_result

### Sales Performance by Branch
This query calculates the total sales for each branch and displays them in descending order of sales.

In [7]:
# Sales Performance by Branch
branch_performance_query = """
SELECT branch, SUM(total) AS total_sales
FROM sales
GROUP BY branch
ORDER BY total_sales DESC;
"""
cursor.execute(branch_performance_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

  branch total_sales
0      C   110568.86
1      A   106200.57
2      B   106198.00


### Average Rating by Product Line
This query calculates the average customer rating for each product line.

In [8]:
# Average Rating by Product Line
product_line_rating_query = """
SELECT product_line, AVG(rating) AS avg_rating
FROM sales
GROUP BY product_line
ORDER BY avg_rating DESC;
"""
cursor.execute(product_line_rating_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

             product_line avg_rating
0      Food and beverages   7.113218
1     Fashion accessories   7.029213
2       Health and beauty   7.003289
3  Electronic accessories   6.924706
4       Sports and travel   6.916265
5      Home and lifestyle   6.837500


### Daily Sales
This query provides a summary of daily sales, including the total sales amount, total sales units, and the average item price.

In [9]:
# Sales for each day
daily_sales_query = """
SELECT 
    date AS sales_date, 
    SUM(total) AS total_sales_amount, 
    SUM(quantity) AS total_sales_units, 
    SUM(total) / SUM(quantity) AS avg_item_price
FROM sales
GROUP BY date
ORDER BY date;
"""
cursor.execute(daily_sales_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

    sales_date total_sales_amount total_sales_units avg_item_price
0   2019-01-01            4745.19                81      58.582593
1   2019-01-02            1945.50                48      40.531250
2   2019-01-03            2078.12                37      56.165405
3   2019-01-04            1623.68                32      50.740000
4   2019-01-05            3536.69                55      64.303455
..         ...                ...               ...            ...
84  2019-03-26            1962.52                52      37.740769
85  2019-03-27            2902.81                45      64.506889
86  2019-03-28            2229.42                48      46.446250
87  2019-03-29            4023.25                54      74.504630
88  2019-03-30            4487.06                67      66.971045

[89 rows x 4 columns]


### Monthly Sales
This query provides a summary of monthly sales, including the total sales amount, total sales units, and the average item price.

In [10]:
# Sales for each month
monthly_sales_query = """
SELECT 
    DATE_FORMAT(date, '%Y-%m') AS sales_month, 
    SUM(total) AS total_sales_amount,
    SUM(quantity) AS total_sales_units,
    SUM(total) / SUM(quantity) AS avg_item_price
FROM sales
GROUP BY DATE_FORMAT(date, '%Y-%m')
ORDER BY DATE_FORMAT(date, '%Y-%m');
"""
cursor.execute(monthly_sales_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

  sales_month total_sales_amount total_sales_units avg_item_price
0     2019-01          116292.11              1965      59.181735
1     2019-02           97219.58              1654      58.778464
2     2019-03          109455.74              1891      57.882464


### Weekly Sales
This query provides a summary of weekly sales, including the total sales amount, total sales units, and the average item price.

In [11]:
# Sales for each week
weekly_sales_query = """
SELECT 
    YEARWEEK(date, 1) AS sales_week, 
    SUM(total) AS total_sales_amount,
    SUM(quantity) AS total_sales_units,
    SUM(total) / SUM(quantity) AS avg_item_price
FROM sales
GROUP BY YEARWEEK(date, 1)
ORDER BY YEARWEEK(date, 1);
"""
cursor.execute(weekly_sales_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

    sales_week total_sales_amount total_sales_units avg_item_price
0       201901           17543.40               305      57.519344
1       201902           24461.26               431      56.754664
2       201903           28693.43               461      62.241714
3       201904           29286.95               489      59.891513
4       201905           28360.53               484      58.596136
5       201906           27101.89               506      53.561047
6       201907           25563.65               399      64.069298
7       201908           17328.70               314      55.186943
8       201909           29219.75               465      62.838172
9       201910           28418.94               496      57.296250
10      201911           23990.24               447      53.669441
11      201912           25120.66               405      62.026321
12      201913           17878.03               308      58.045552


### Customer Preferences by Payment Method
This query analyzes customer preferences by the payment method, showing the number of transactions and total sales amount for each method.

In [12]:
# Customer Preferences by Payment Method
payment_method_query = """
SELECT payment, COUNT(*) AS number_of_transactions, SUM(total) AS total_sales_amount
FROM sales
GROUP BY payment
ORDER BY number_of_transactions DESC;
"""
cursor.execute(payment_method_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

       payment  number_of_transactions total_sales_amount
0      Ewallet                     345          109993.38
1         Cash                     344          112206.76
2  Credit card                     311          100767.29


### Average Revenue per Customer Type
This query calculates the average revenue per customer type (members vs. normal customers).

In [13]:
# The Average Revenue per Customer Type (Members vs. Normal)
revenue_per_customer_type_query = """
WITH CustomerRevenue AS (
    SELECT customer_type, SUM(total) AS total_revenue
    FROM sales
    GROUP BY customer_type
),
CustomerCount AS (
    SELECT customer_type, COUNT(DISTINCT invoice_id) AS total_customers
    FROM sales
    GROUP BY customer_type
)
SELECT 
    cr.customer_type, cr.total_revenue, cc.total_customers,
    cr.total_revenue / cc.total_customers AS avg_revenue_per_customer
FROM CustomerRevenue cr
JOIN CustomerCount cc ON cr.customer_type = cc.customer_type;
"""
cursor.execute(revenue_per_customer_type_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

  customer_type total_revenue  total_customers avg_revenue_per_customer
0        Member     164223.81              501               327.792036
1        Normal     158743.62              499               318.123487


### Sales Patterns by Day and Time
This query analyzes sales patterns across different days of the week and times of the day.

In [14]:
# Sales Patterns across different days of the week and times of the day
sales_patterns_query = """
WITH SalesWithWeekDay AS (
    SELECT *, DAYOFWEEK(date) AS weekday
    FROM sales
)
SELECT weekday, HOUR(time) AS hour, SUM(total) AS total_sales_amount
FROM SalesWithWeekDay
GROUP BY weekday, hour
ORDER BY weekday, hour;
"""
cursor.execute(sales_patterns_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

    weekday  hour total_sales_amount
0         1    10            4067.35
1         1    11            4736.78
2         1    12            4728.22
3         1    13            5159.58
4         1    14            4676.13
..      ...   ...                ...
72        7    16            3813.51
73        7    17            5069.82
74        7    18            6738.75
75        7    19            9117.40
76        7    20            3185.06

[77 rows x 3 columns]


### Customer Segmentation by Spending
This query segments customers based on their spending into high spenders, medium spenders, and low spenders.

In [15]:
# Customer Segmentation Analysis based on their spending
customer_segmentation_query = """
WITH CustomerSpending AS (
    SELECT branch, customer_type, gender, SUM(total) AS total_spending
    FROM sales
    GROUP BY branch, customer_type, gender
)
SELECT branch, customer_type, gender, total_spending,
    CASE
        WHEN total_spending > 30000 THEN 'High Spender'
        WHEN total_spending BETWEEN 25000 AND 30000 THEN 'Medium Spender'
        ELSE 'Low Spender'
    END AS spending_segment
FROM CustomerSpending
ORDER BY total_spending DESC;
"""
cursor.execute(customer_segmentation_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

   branch customer_type  gender total_spending spending_segment
0       C        Member  Female       34653.51     High Spender
1       C        Normal  Female       27032.05   Medium Spender
2       A        Member    Male       26994.49   Medium Spender
3       B        Member    Male       26854.30   Medium Spender
4       B        Member  Female       26850.53   Medium Spender
5       C        Normal    Male       26655.40   Medium Spender
6       A        Member  Female       26643.08   Medium Spender
7       A        Normal  Female       26626.16   Medium Spender
8       B        Normal    Male       26415.24   Medium Spender
9       B        Normal  Female       26077.93   Medium Spender
10      A        Normal    Male       25936.84   Medium Spender
11      C        Member    Male       22227.90      Low Spender


### Sales Forecasting
This query forecasts future sales using a moving average window function.

In [16]:
# Forecasting Future Sales using a window function AVG()
forecasting_sales_query = """
WITH DailySales AS (
    SELECT date, SUM(total) AS daily_total_sales
    FROM sales
    GROUP BY date
),
MovingAverage AS (
    SELECT date, daily_total_sales,
        AVG(daily_total_sales) OVER (ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS windowed_avg_week
    FROM DailySales
)
SELECT date, daily_total_sales, windowed_avg_week
FROM MovingAverage
ORDER BY date;
"""
cursor.execute(forecasting_sales_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

          date daily_total_sales windowed_avg_week
0   2019-01-01           4745.19       4745.190000
1   2019-01-02           1945.50       3345.345000
2   2019-01-03           2078.12       2922.936667
3   2019-01-04           1623.68       2598.122500
4   2019-01-05           3536.69       2785.836000
..         ...               ...               ...
84  2019-03-26           1962.52       3188.984286
85  2019-03-27           2902.81       2823.927143
86  2019-03-28           2229.42       2874.198571
87  2019-03-29           4023.25       2994.784286
88  2019-03-30           4487.06       3050.787143

[89 rows x 3 columns]


### Sales Anomalies
This query identifies days with unusually high or low sales based on statistical analysis.

In [17]:
# Anomaly in Sales (Identifying days with unusually high or low sales)
sales_anomaly_query = """
WITH DailySales AS (
    SELECT date, SUM(total) AS daily_total_sales
    FROM sales
    GROUP BY date
),
SalesStats AS (
    SELECT AVG(daily_total_sales) AS avg_sales, STDDEV(daily_total_sales) AS stddev_sales
    FROM DailySales
)
SELECT 
    ds.date, ds.daily_total_sales, ss.avg_sales, ss.stddev_sales,
    CASE
        WHEN ds.daily_total_sales > ss.avg_sales + 2 * ss.stddev_sales THEN 'High'
        WHEN ds.daily_total_sales < ss.avg_sales - 2 * ss.stddev_sales THEN 'Low'
        ELSE 'None'
    END AS anomaly_type
FROM DailySales ds
CROSS JOIN SalesStats ss 
WHERE ds.daily_total_sales > ss.avg_sales + 2 * ss.stddev_sales 
   OR ds.daily_total_sales < ss.avg_sales - 2 * ss.stddev_sales
ORDER BY ds.date;
"""
cursor.execute(sales_anomaly_query)
result = cursor.fetchall()

# To display the results in a DataFrame, so that it is easier to read
df_result = query_to_dataframe(cursor, result)

         date daily_total_sales    avg_sales  stddev_sales anomaly_type
0  2019-02-07           7228.22  3628.847528   1519.034524         High
1  2019-02-15           6830.79  3628.847528   1519.034524         High
2  2019-03-09           7474.05  3628.847528   1519.034524         High
3  2019-03-14           7214.63  3628.847528   1519.034524         High


## The End of the Session

In [18]:
# Close the cursor and connection
if cursor:
    cursor.close()
if connection:
    connection.close()